<a href="https://colab.research.google.com/github/OneFineStarstuff/State-of-the-Art/blob/main/Pyro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pyro-ppl

In [ ]:
import pyro
import pyro.distributions as dist
import torch

# Define the model
def model(data):
    loc = pyro.sample("loc", dist.Normal(0., 1.))
    scale = pyro.sample("scale", dist.LogNormal(0., 1.))
    with pyro.plate("data", len(data)):
        pyro.sample("obs", dist.Normal(loc, scale), obs=data)

# Define the guide
def guide(data):
    loc_loc = pyro.param("loc_loc", torch.tensor(0.))
    loc_scale = pyro.param("loc_scale", torch.tensor(1.))
    scale_loc = pyro.param("scale_loc", torch.tensor(0.))
    scale_scale = pyro.param("scale_scale", torch.tensor(1.))

    pyro.sample("loc", dist.Normal(loc_loc, loc_scale))
    pyro.sample("scale", dist.LogNormal(scale_loc, scale_scale))

# Generate some data
data = torch.randn(100)

# Clear the parameter store
pyro.clear_param_store()

# Setup the SVI object
svi = pyro.infer.SVI(model=model, guide=guide, optim=pyro.optim.Adam({"lr": 0.01}), loss=pyro.infer.Trace_ELBO())

# Perform inference
num_iterations = 1000
for _ in range(num_iterations):
    loss = svi.step(data)

# Print the learned parameters
for name, value in pyro.get_param_store().items():
    print(name, value)